In [1]:

import regex as re
import pandas as pd
import numpy as np
from termcolor import colored
from urllib.request import urlopen
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
%pwd 
%cd c:\\Users\\Lenovo\\Flask apps\\House-price-prediction\\

c:\Users\Lenovo\Flask apps\House-price-prediction


In [15]:
MAIN_DF_PATH = r'Dataset\calgary_cleaned_rentals_dataset.csv'
df = pd.read_csv(MAIN_DF_PATH)

In [16]:

df.dropna(subset=['price', 'sq_feet'], inplace=True)
df.reset_index(inplace=True, drop=True)

assert len(df.rented.unique()) ==1
df.drop(columns=['rented'], inplace=True)


In [17]:
# handling second rentals --ADD
def handle_second_rentals(df, info_cols = ['price', 'sq_feet', 'beds', 'baths']):
    """
    separates residence options that has two listings
    creates the second dataset
    then renames both dataset's cols to have info_cols names
    """
    info_cols_2 = [col + '2' for col in info_cols]

    # check for where at least two second house columns are not null
    df_second = df[(~df[info_cols_2[0]].isna())&(~df[info_cols_2[1]].isna())]

    df_second = df_second.drop(columns=info_cols)
    for col in info_cols:
        df_second = df_second.rename(columns={col + '2': col})

    df = df.drop(columns=info_cols_2)
    return df, df_second

In [18]:
df, df_second = handle_second_rentals(df, info_cols = ['price', 'sq_feet', 'beds', 'baths'])

In [19]:
df_total = pd.concat([df, df_second], axis=0)
df_total.reset_index(inplace=True, drop=True)

In [20]:
len(df_total)

2213

In [21]:
df_total.head(1)

,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet
0,Immediate,2000-01-01,Calgary,Cliff Bungalow,51.036696,-114.074492,/ab/calgary/rentals/apartment/studio/cliff-bungalow/357976,Apartment,525.0,studio,420,1,0,0,1,1,1,0,0


In [13]:
df_total.columns

Index(['availability', 'a', 'city', 'community', 'latitude', 'longitude',
       'link', 'type', 'cats', 'dogs', 'sq_feet2', 'price2', 'beds2', 'baths2',
       'utility_heat', 'utility_electricity', 'utility_water', 'utility_cable',
       'utility_internet', 'sq_feet', 'price', 'beds', 'baths'],
      dtype='object')

In [53]:
len(df[(~df.price2.isna())&(~df.sq_feet2.isna())&((~df.beds.isna()))&(~df.baths2.isna())])

180

In [29]:
# --ADD
def clean_undefined_dates(df, null_substitute = 'call for availability'):
    """
    call for availability is represented as 3000-01-01 in the dataset
    so we replace these dates with a string
    """
    for i in range(len(df)):
        try: 
            df.loc[i, 'a'] = pd.to_datetime(df.loc[i, 'a'], format='%Y-%m-%d')
        except: 
            df.loc[i, 'a'] = null_substitute
            df.loc[i, 'availability'] = null_substitute
    df.reset_index(inplace=True)
    return df


df = clean_undefined_dates(df)

In [35]:
df[df.sq_feet.isna()][['sq_feet', 'sq_feet2', 'link', 'type', 'price']].head()

,sq_feet,sq_feet2,link,type,price
0,NaN,NaN,/ab/calgary/rentals/apartment/1-bedroom/manchester/468894,Apartment,580.0
1,NaN,NaN,/ab/calgary/rentals/apartment/studio/downtown/472999,Apartment,580.0
4,NaN,NaN,/ab/calgary/rentals/apartment/1-bedroom/downtown/477769,Apartment,960.0
6,NaN,NaN,/ab/calgary/rentals/apartment/1-bedroom/beltline/452842,Apartment,925.0
8,NaN,NaN,/ab/calgary/rentals/apartment/2-bedrooms/banff-trail/pet-friendly/403379,Apartment,940.0


In [43]:
df[['price', 'price2']]

,price,price2
0,999.0,NaN
1,974.0,999.0
2,899.0,950.0
3,849.0,999.0
4,800.0,1000.0
...,...,...
182,1245.0,2300.0
183,2700.0,2800.0
184,1567.0,1839.0
185,2925.0,3650.0


In [27]:
df[df.availability != 'Immediate'].availability.unique()
#df.loc[:, 'availability_month'] 

array(['May 01', 'Apr 15', 'Apr 30', 'call for availability', 'May 15',
       'Apr 07', 'Apr 17', 'Apr 20', 'Apr 21', 'Apr 28', 'May 06',
       'May 07', 'May 09', 'May 10', 'May 13', 'May 20', 'Jun 01',
       'Jun 15', 'Jul 01', 'Jul 15', 'Aug 01', 'Sep 01', 'Apr 13',
       'Apr 14', 'Jun 30', 'Nov 01', 'Apr 11', 'Apr 09', 'Apr 10',
       'Apr 12', 'Apr 16', 'May 26', 'Apr 08', 'May 05', 'May 31',
       'Jul 08', 'Oct 01', 'Apr 18', 'May 02', 'May 16', 'Jun 05',
       'Sep 24', 'Jun 20', 'Jun 22', 'Jun 02', 'Apr 25', 'May 03',
       'May 18', 'May 21', 'May 28', 'Jun 07', 'Jun 16'], dtype=object)